In [ ]:
# load packages
%pip install pandas
import pandas as pd
import numpy as np
%pip install langdetect
from langdetect import detect
from ast import literal_eval
%pip install nltk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
%pip install spacy
import spacy
from tqdm import tqdm
%pip install sklearn
from sklearn.feature_extraction.text import CountVectorizer
%pip install yake
import yake

from tqdm import tqdm
tqdm.pandas()



In [3]:
df = pd.read_csv(r'C:\Users\jason\Downloads\data.csv',names= ['Vendor Name','Homepage Text'])
df

,Vendor Name,Homepage Text
0,prnews.io,PRNEWS.IO — Content Marketing Platform With To...
1,proficrm.com,"CRM Software - Easy to use, modern and efficie..."
2,atollon.com,Home | Atollon Přejít k hlavnímu obsahu Platfo...
3,dinamikcrm.com,İşinizi daha kolay ve verimli yönetmek istemez...
4,nectarcrm.com.br,Nectar | Software CRM para Gestão de Vendas B2...
...,...,...
8939,overmore.com,LEADSCALE - Overmore Redirect WE HAVE REBRANDE...
8940,velocity-it.com,Digital Transformation & IT Automation | Veloc...
8941,synkd.life,"Synkd If you're seeing this message, that mean..."
8942,zymplify.com,"Zymplify - Revenue Acceleration, Operations & ..."


In [4]:
# tokenization
import string
def tokenize(processed_texts = [], remove_punctuation=True, 
             to_lowercase=True, stop_words=[], word_types=[], to_lemma=True):
    
    """
    Tokenizes the text entries in df_column using spacy tokenizer 
    and filters this on supplied arguments, or processes the already
    tokenized texts of processed_texts further
    
    Parameters
    ----------
    df_column : pandas Series object (default None)
        Series with texts that need to be tokenized.
        If None is supplied, then processed_texts needs to be given to process that
        one further using the other arguments instead of tokenizing df_column from the start
    processed_texts : list of list of str (default [])
        List of tokenized texts that need to be further processed using the other arguments.
        If this is supplied, then df_column needs to be None
    remove_punctuation : bool (default True)
        Boolean stating if punctuation needs to be removed from the tokenized texts
    to_lowercase : bool (default True)
        Boolean stating if the tokenized texts need to be lowercased
    stop_words : list of str (default [])
        List with (lowercased) stop words to remove from the tokenized texts,
        with the default being no words to remove
    word_types : list of str (default [])
        List with word types to keep from the tokenized texts, which have to be
        one or multiple of the POS tags of Spacy (e.g. see here for examples
        https://machinelearningknowledge.ai/tutorial-on-spacy-part-of-speech-pos-tagging/)
    to_lemma : bool (default True)
        Boolean stating if the tokenized texts need to be lemmatized

        
    Returns
    -------
    raw_processed_texts : list of lists of str
        List with a list of tokenized words per data entry in df_column.
        This is a raw processed texts without taking the other arguments into
        account, such that still multiple different processing methods on the tokens can
        be done without needing to tokenize the full df_column anew
    processed_texts : list of lists of str
        List with a list of tokenized words per data entry in df_column. 
        This is the fully processed texts that take all arguments into account
        to reach the desired tokenization result
    """
    #print(processed_texts)
    
    
    nlp = spacy.load("en_core_web_sm")
    # Remove spaces that are presented as an empty string 
    processed_text = nlp(processed_texts)
    
   
    
    
    processed_text = [word for word in processed_text if not word.text.isspace()]
    
    raw_processed_texts = processed_text.copy()
    #print(raw_processed_texts)
    
    # Remove punctuation if wanted
    if remove_punctuation:
        processed_text = [word for word in processed_text if not word.is_punct] 
        
    # Remove the given stopwords # TODO [] empty list is standard
    if stop_words is not []:
        processed_text = [word for word in processed_text if not word.text in stop_words] 
                           
        
    # Filter on given word types, e.g. proper nouns ('PROPN'), verbs ('VERB'), nouns ('NOUN') and adjectives ('ADJ')  
    if word_types is not []:
        processed_text = [word for word in processed_text if word.pos_ in word_types] 
                           
    
    # Keep only lemmatized words if wanted
    if to_lemma:
        processed_text = [word.lemma_ for word in processed_text] 
                           
        
    # Lowercase the text if wanted. When it is lemmatized already, then each item is a string so directly
    # apply .lower() on it, and if it still is a token, then need to apply .text.lower().
    # Perform this as last step because the previous steps still require the spacy tokens
    if to_lowercase:
        processed_text = [word.lower() if isinstance(word, str) else word.text.lower() for word in processed_text] 
                           
    
    
    return raw_processed_texts, processed_text

In [ ]:
import random
import wikipedia
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer



# Specify the title of the Wikipedia page
df_wiki_pages=pd.read_csv(r'C:\Users\jason\Downloads\Categories.csv',usecols=[0])


df_wiki_pages = df_wiki_pages.dropna()
#print(df_wiki_pages)
wikipages = []
#for pages in wikipages:

tokenized_texts = []
for index, row in df_wiki_pages.iterrows():
    currentRow = row['Categories']
    print("Now testing: "+ currentRow)
    wikisearch = wikipedia.search(currentRow)
    print(wikisearch)
    try:
        wiki = wikipedia.page(wikisearch[0],auto_suggest=False)
    except wikipedia.DisambiguationError as e:
        s = random.choice(e.options)
        wiki = wikipedia.page(s)
    #print(wiki)
    # Extract the plain text content of the page
    text = wiki.content
    country_list = ['Afghanistan', 'Aland Islands', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antarctica', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia, Plurinational State of', 'Bonaire, Sint Eustatius and Saba', 'Bosnia and Herzegovina', 'Botswana', 'Bouvet Island', 'Brazil', 'British Indian Ocean Territory', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Chile', 'China', 'Christmas Island', 'Cocos (Keeling) Islands', 'Colombia', 'Comoros', 'Congo', 'Congo, The Democratic Republic of the', 'Cook Islands', 'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba', 'Curaçao', 'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Ethiopia', 'Falkland Islands (Malvinas)', 'Faroe Islands', 'Fiji', 'Finland', 'France', 'French Guiana', 'French Polynesia', 'French Southern Territories', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Gibraltar', 'Greece', 'Greenland', 'Grenada', 'Guadeloupe', 'Guam', 'Guatemala', 'Guernsey', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Heard Island and McDonald Islands', 'Holy See (Vatican City State)', 'Honduras', 'Hong Kong', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran, Islamic Republic of', 'Iraq', 'Ireland', 'Isle of Man', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jersey', 'Jordan', 'Kazakhstan', 'Kenya', 'Kiribati', "Korea, Democratic People's Republic of", 'Korea, Republic of', 'Kuwait', 'Kyrgyzstan', "Lao People's Democratic Republic", 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Macao', 'Macedonia, Republic of', 'Madagascar', 'Malawi', 'Malaysia', 'Maldives', 'Mali', 'Malta', 'Marshall Islands', 'Martinique', 'Mauritania', 'Mauritius', 'Mayotte', 'Mexico', 'Micronesia, Federated States of', 'Moldova, Republic of', 'Monaco', 'Mongolia', 'Montenegro', 'Montserrat', 'Morocco', 'Mozambique', 'Myanmar', 'Namibia', 'Nauru', 'Nepal', 'Netherlands', 'New Caledonia', 'New Zealand', 'Nicaragua', 'Niger', 'Nigeria', 'Niue', 'Norfolk Island', 'Northern Mariana Islands', 'Norway', 'Oman', 'Pakistan', 'Palau', 'Palestinian Territory, Occupied', 'Panama', 'Papua New Guinea', 'Paraguay', 'Peru', 'Philippines', 'Pitcairn', 'Poland', 'Portugal', 'Puerto Rico', 'Qatar', 'Réunion', 'Romania', 'Russian Federation', 'Rwanda', 'Saint Barthélemy', 'Saint Helena, Ascension and Tristan da Cunha', 'Saint Kitts and Nevis', 'Saint Lucia', 'Saint Martin (French part)', 'Saint Pierre and Miquelon', 'Saint Vincent and the Grenadines', 'Samoa', 'San Marino', 'Sao Tome and Principe', 'Saudi Arabia', 'Senegal', 'Serbia', 'Seychelles', 'Sierra Leone', 'Singapore', 'Sint Maarten (Dutch part)', 'Slovakia', 'Slovenia', 'Solomon Islands', 'Somalia', 'South Africa', 'South Georgia and the South Sandwich Islands', 'Spain', 'Sri Lanka', 'Sudan', 'Suriname', 'South Sudan', 'Svalbard and Jan Mayen', 'Swaziland', 'Sweden', 'Switzerland', 'Syrian Arab Republic', 'Taiwan, Province of China', 'Tajikistan', 'Tanzania, United Republic of', 'Thailand', 'Timor-Leste', 'Togo', 'Tokelau', 'Tonga', 'Trinidad and Tobago', 'Tunisia', 'Turkey', 'Turkmenistan', 'Turks and Caicos Islands', 'Tuvalu', 'Uganda', 'Ukraine', 'United Arab Emirates', 'United Kingdom', 'United States', 'United States Minor Outlying Islands', 'Uruguay', 'Uzbekistan', 'Vanuatu', 'Venezuela, Bolivarian Republic of', 'Viet Nam', 'Virgin Islands, British', 'Virgin Islands, U.S.', 'Wallis and Futuna', 'Yemen', 'Zambia', 'Zimbabwe', 'Asia', 'Europe', 'Africa', 'America', 'Australia', 'Antarctica', 'Arab', 'Emirates', 'Korea', 'British', 'Dutch', 'Republic', 'Island']
    country_list = [i.lower() for i in country_list]

    stopword_list = stopwords.words('english') + country_list 

    #print(tokenize(text, to_lowercase = True, remove_punctuation = True, word_types = ['PROPN', 'VERB', 'NOUN', 'ADJ'], to_lemma=False, stop_words = stopword_list)[1])

    tokenized_text = tokenize(text, to_lowercase = True, remove_punctuation = True, word_types = ['PROPN', 'VERB', 'NOUN', 'ADJ'], to_lemma=False, stop_words = stopword_list)[1]
    tokenized_texts.append(tokenized_text)
df_wiki_pages["Webpage Tokenized Text"] = tokenized_texts
df_wiki_pages


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf_vectorizer_wiki = TfidfVectorizer(use_idf=True, max_df=.8, min_df = 5,
                        ngram_range=(1,3), 
                        stop_words='english')

out = ' '.join(tokenized_text)

DF = {}
for i in range(len(tokenized_text)):
    tokens = tokenized_text[i]
    #print(tokens)
    try:
        DF[tokens].add(i)
    except:
        DF[tokens] = {i}
#print(DF)

for i in DF:
    DF[i] = len(DF[i])
print(DF)


{'computer': 2, 'programming': 7, 'language': 11, 'implementation': 6, 'system': 1, 'executing': 1, 'programs': 1, 'general': 1, 'approaches': 1, 'interpretation': 2, 'program': 10, 'read': 3, 'input': 3, 'interpreter': 6, 'performs': 1, 'actions': 1, 'written': 5, 'compilation': 4, 'compiler': 13, 'translates': 2, 'bytecode': 9, 'machine': 13, 'code': 7, 'translated': 1, 'executed': 3, 'hardware': 2, 'serve': 1, '=': 16, 'composed': 1, 'parts': 1, 'parser': 3, 'evaluator': 2, 'processed': 1, 'breaks': 1, 'components': 1, 'form': 1, 'parse': 2, 'tree': 2, 'uses': 1, 'execute': 1, 'virtual': 9, 'special': 1, 'type': 1, 'interprets': 1, 'portable': 3, 'low': 1, 'level': 3, 'similar': 1, 'physical': 1, 'improve': 2, 'efficiencies': 1, 'many': 2, 'languages': 5, 'java': 2, 'python': 3, 'c': 2, 'compiled': 2, 'interpreted': 2, 'time': 4, 'machines': 1, 'include': 3, 'jit': 4, 'efficiency': 1, 'execution': 2, 'determines': 1, 'portion': 2, 'used': 4, 'compiles': 1, 'particular': 1, 'stores':

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [out]

tr_idf_model  = TfidfVectorizer()
tf_idf_vector = tr_idf_model.fit_transform(corpus)
tf_idf_array = tf_idf_vector.toarray()

#print(tf_idf_array)

words_set = tr_idf_model.get_feature_names_out()

#print(words_set)
df_tf_idf = pd.DataFrame(tf_idf_array, columns = words_set)

df_tf_idf

print(df_tf_idf.values.tolist())
print(df_tf_idf.columns.values.tolist())

[[0.02709639073017024, 0.02709639073017024, 0.05419278146034048, 0.02709639073017024, 0.02709639073017024, 0.2438675165715322, 0.02709639073017024, 0.1896747351111917, 0.02709639073017024, 0.02709639073017024, 0.10838556292068097, 0.02709639073017024, 0.05419278146034048, 0.35225307949221313, 0.05419278146034048, 0.02709639073017024, 0.02709639073017024, 0.02709639073017024, 0.02709639073017024, 0.05419278146034048, 0.02709639073017024, 0.02709639073017024, 0.02709639073017024, 0.02709639073017024, 0.02709639073017024, 0.02709639073017024, 0.10838556292068097, 0.02709639073017024, 0.02709639073017024, 0.02709639073017024, 0.10838556292068097, 0.05419278146034048, 0.05419278146034048, 0.02709639073017024, 0.02709639073017024, 0.08128917219051073, 0.02709639073017024, 0.05419278146034048, 0.02709639073017024, 0.02709639073017024, 0.02709639073017024, 0.02709639073017024, 0.02709639073017024, 0.02709639073017024, 0.02709639073017024, 0.05419278146034048, 0.02709639073017024, 0.02709639073

In [8]:

def Nmaxelements(list1,list2, N):
    final_list = []
    final_list2=[]
    final_dict = {}
    for i in range(0, N):
        max1 = 0
        max2 = ""
        
        for j in range(len(list1)):	
            if list1[j] > max1:
                max1 = list1[j]
                max2 = list2[j]
            
                
        list1.remove(max1)
        #final_list.append(max1)
        final_dict[max2] = max1
        #final_list2.append(max2)		
    #print(final_list)
    #print(final_list2)
    return final_dict

In [9]:
print(Nmaxelements(df_tf_idf.values.tolist()[0],df_tf_idf.columns.values.tolist(),40).keys())

dict_keys(['compiler', 'low', 'jython', 'processed', 'bytecode', 'understanding', 'called', 'portion', 'given', 'input', 'interpreter', 'transpiler', 'combinations', 'data', 'development', 'run', 'strike', 'elements', 'general', 'improves', 'interpretation', 'linking', 'machines', 'most', 'output', 'back', 'compiled', 'cpython', 'determines', 'evaluator', 'executable'])


In [29]:
import yake
kw_extractor = yake.KeywordExtractor()
# select test text
all_keywords = []
for index, row in df.iterrows():
     currentRow = row['Homepage Text']
     text = currentRow
     #print(text)
     language = "en"
     max_ngram_size = 3
     deduplication_threshold = 0.9 #0.9 allows for the same words in different keywords, 0.1 prevents this
     numOfKeywords = 50
     custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
     keywords = custom_kw_extractor.extract_keywords(text)
     #print(keywords)
     keys = []
     for kw in keywords:
          #print(kw[0])
          keys.append(kw[0])
     all_keywords.append(keys)
     
df_wiki_pages["Extracted Keywords"] = all_keywords
     

Warning! Exception: object of type 'float' has no len() generated by the following text: 'nan' 
Warning! Exception: object of type 'float' has no len() generated by the following text: 'nan' 
Warning! Exception: object of type 'float' has no len() generated by the following text: 'nan' 
Warning! Exception: object of type 'float' has no len() generated by the following text: 'nan' 
Warning! Exception: object of type 'float' has no len() generated by the following text: 'nan' 
Warning! Exception: object of type 'float' has no len() generated by the following text: 'nan' 
Warning! Exception: object of type 'float' has no len() generated by the following text: 'nan' 
Warning! Exception: object of type 'float' has no len() generated by the following text: 'nan' 
Warning! Exception: object of type 'float' has no len() generated by the following text: 'nan' 
Warning! Exception: object of type 'float' has no len() generated by the following text: 'nan' 
Warning! Exception: object of type 'floa

ValueError: Length of values (8944) does not match length of index (226)

In [11]:
df_wiki_pages

,Categories,Webpage Tokenized Text,Extracted Keywords
0,Personalized_marketing,"[personalized, marketing, known, marketing, in...","[personalized marketing adopted, forms social ..."
1,Customer_relationship_management,"[customer, relationship, management, crm, proc...","[customer relationship management, customer re..."
2,Customer_experience,"[customer, experience, cx, totality, cognitive...","[customer experience management, customer expe..."
3,Social_media_marketing,"[social, media, marketing, use, social, media,...","[social media marketing, social media, social ..."
4,Content_marketing,"[content, marketing, form, marketing, focused,...","[digital content marketing, time spent page, o..."
...,...,...,...
221,Stock_photography,"[stock, photography, supply, photographs, lice...","[stock photo agencies, images stock agencies, ..."
222,Bidding,"[bidding, offer, competitive, set, price, tag,...","[bidding process students, bid construction pr..."
223,Certification,"[certification, provision, independent, body, ...","[party attestations conformity, international ..."
224,Message_sequence_chart,"[message, sequence, chart, msc, interaction, d...","[message sequence chart, live sequence charts,..."


In [12]:
df_wiki_pages["Extracted Keywords String"] = df_wiki_pages['Extracted Keywords'].apply(lambda x:" ".join(x))
df_wiki_pages

,Categories,Webpage Tokenized Text,Extracted Keywords,Extracted Keywords String
0,Personalized_marketing,"[personalized, marketing, known, marketing, in...","[personalized marketing adopted, forms social ...",personalized marketing adopted forms social me...
1,Customer_relationship_management,"[customer, relationship, management, crm, proc...","[customer relationship management, customer re...",customer relationship management customer rela...
2,Customer_experience,"[customer, experience, cx, totality, cognitive...","[customer experience management, customer expe...",customer experience management customer experi...
3,Social_media_marketing,"[social, media, marketing, use, social, media,...","[social media marketing, social media, social ...",social media marketing social media social net...
4,Content_marketing,"[content, marketing, form, marketing, focused,...","[digital content marketing, time spent page, o...",digital content marketing time spent page onli...
...,...,...,...,...
221,Stock_photography,"[stock, photography, supply, photographs, lice...","[stock photo agencies, images stock agencies, ...",stock photo agencies images stock agencies mod...
222,Bidding,"[bidding, offer, competitive, set, price, tag,...","[bidding process students, bid construction pr...",bidding process students bid construction prob...
223,Certification,"[certification, provision, independent, body, ...","[party attestations conformity, international ...",party attestations conformity international so...
224,Message_sequence_chart,"[message, sequence, chart, msc, interaction, d...","[message sequence chart, live sequence charts,...",message sequence chart live sequence charts se...


In [17]:
import yake
kw_extractor = yake.KeywordExtractor()
# select test text
all_keywords = []
for index, row in df.iterrows():
     currentRow = row['Homepage Text']
     text = currentRow
     #print(text)
     language = "en"
     max_ngram_size = 3
     deduplication_threshold = 0.9 #0.9 allows for the same words in different keywords, 0.1 prevents this
     numOfKeywords = 50
     custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
     keywords = custom_kw_extractor.extract_keywords(text)
     #print(keywords)
     keys = []
     for kw in keywords:
          #print(kw[0])
          keys.append(kw[0])
     all_keywords.append(keys)
     
df["Extracted Keywords"] = all_keywords
     

Warning! Exception: object of type 'float' has no len() generated by the following text: 'nan' 
Warning! Exception: object of type 'float' has no len() generated by the following text: 'nan' 
Warning! Exception: object of type 'float' has no len() generated by the following text: 'nan' 
Warning! Exception: object of type 'float' has no len() generated by the following text: 'nan' 
Warning! Exception: object of type 'float' has no len() generated by the following text: 'nan' 
Warning! Exception: object of type 'float' has no len() generated by the following text: 'nan' 
Warning! Exception: object of type 'float' has no len() generated by the following text: 'nan' 
Warning! Exception: object of type 'float' has no len() generated by the following text: 'nan' 
Warning! Exception: object of type 'float' has no len() generated by the following text: 'nan' 
Warning! Exception: object of type 'float' has no len() generated by the following text: 'nan' 
Warning! Exception: object of type 'floa

In [18]:
df

,Vendor Name,Homepage Text,Predicted Category,Extracted Keywords
0,prnews.io,PRNEWS.IO — Content Marketing Platform With To...,"[(Personalized_marketing, 0.004314358445064406...","[Store Blog Sign, Version Online Store, conten..."
1,proficrm.com,"CRM Software - Easy to use, modern and efficie...","[(Personalized_marketing, 0.004402432161744197...","[confirmation link, Login SIGNUP FREE, CRM Sof..."
2,atollon.com,Home | Atollon Přejít k hlavnímu obsahu Platfo...,"[(Personalized_marketing, 0.004356150538710852...","[Blog Cena Kontakt, Cena Kontakt Select, Home ..."
3,dinamikcrm.com,İşinizi daha kolay ve verimli yönetmek istemez...,"[(Personalized_marketing, 0.004336341041280748...","[yönetmek istemez misiniz, verimli yönetmek is..."
4,nectarcrm.com.br,Nectar | Software CRM para Gestão de Vendas B2...,"[(Personalized_marketing, 0.004406642291844435...","[Grátis Menu Menu, Teste Grátis Menu, Entrar T..."
...,...,...,...,...
8939,overmore.com,LEADSCALE - Overmore Redirect WE HAVE REBRANDE...,"[(Personalized_marketing, 0.004454100471426761...","[Overmore Redirect, Click here Linkedin, leads..."
8940,velocity-it.com,Digital Transformation & IT Automation | Veloc...,"[(Personalized_marketing, 0.004339655002166887...","[Business Process Automation, Robotic Process ..."
8941,synkd.life,"Synkd If you're seeing this message, that mean...","[(Personalized_marketing, 0.0044217612038575),...","[app work, make this app, Synkd, message, brow..."
8942,zymplify.com,"Zymplify - Revenue Acceleration, Operations & ...","[(Personalized_marketing, 0.004267136083555218...","[Revenue Operations Marketing, Intent Revenue ..."


In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline


model = make_pipeline(TfidfVectorizer(),MultinomialNB())
model.fit(df_wiki_pages['Extracted Keywords String'],df_wiki_pages['Categories'])
df['Extracted Keywords String'] = df['Extracted Keywords'].apply(lambda x:" ".join(x))

y_pred = model.predict_proba(df['Extracted Keywords String'].values.astype('U'))
print(y_pred)
#print(np.split(y_pred,1))
finalPredictions = []
for index in np.split(y_pred,1)[0]:
    #print(np.split(index,1))
    for value in np.split(index,1):
        tempList = []
        for i in range(0,len(value)):
            tempList.append((df_wiki_pages.loc[i]['Categories'],value[i]))
        finalPredictions.append(tempList)     
    print("New predictions")
df["Predicted Category"] = finalPredictions
df


[[0.00431542 0.00434966 0.0043738  ... 0.00431999 0.00433404 0.00438025]
 [0.00444854 0.00439568 0.00439022 ... 0.00438533 0.00439862 0.00437698]
 [0.00440371 0.00441006 0.00441022 ... 0.00440495 0.00440275 0.0044008 ]
 ...
 [0.00442856 0.00439504 0.0043952  ... 0.00445693 0.00444428 0.00448952]
 [0.00428316 0.00432292 0.00430206 ... 0.00428502 0.00449033 0.00433248]
 [0.00428269 0.00430491 0.00429989 ... 0.0042901  0.00466571 0.00451179]]
New predictions
New predictions
New predictions
New predictions
New predictions
New predictions
New predictions
New predictions
New predictions
New predictions
New predictions
New predictions
New predictions
New predictions
New predictions
New predictions
New predictions
New predictions
New predictions
New predictions
New predictions
New predictions
New predictions
New predictions
New predictions
New predictions
New predictions
New predictions
New predictions
New predictions
New predictions
New predictions
New predictions
New predictions
New predicti

,Vendor Name,Homepage Text,Predicted Category,Extracted Keywords,Extracted Keywords String
0,prnews.io,PRNEWS.IO — Content Marketing Platform With To...,"[(Personalized_marketing, 0.004315417412239644...","[Store Blog Sign, Version Online Store, conten...",Store Blog Sign Version Online Store content o...
1,proficrm.com,"CRM Software - Easy to use, modern and efficie...","[(Personalized_marketing, 0.004448535209032287...","[confirmation link, Login SIGNUP FREE, CRM Sof...",confirmation link Login SIGNUP FREE CRM Softwa...
2,atollon.com,Home | Atollon Přejít k hlavnímu obsahu Platfo...,"[(Personalized_marketing, 0.004403705303672669...","[Blog Cena Kontakt, Cena Kontakt Select, Home ...",Blog Cena Kontakt Cena Kontakt Select Home CRM...
3,dinamikcrm.com,İşinizi daha kolay ve verimli yönetmek istemez...,"[(Personalized_marketing, 0.004415895102354044...","[yönetmek istemez misiniz, verimli yönetmek is...",yönetmek istemez misiniz verimli yönetmek iste...
4,nectarcrm.com.br,Nectar | Software CRM para Gestão de Vendas B2...,"[(Personalized_marketing, 0.004387197681973473...","[Grátis Menu Menu, Teste Grátis Menu, Entrar T...",Grátis Menu Menu Teste Grátis Menu Entrar Test...
...,...,...,...,...,...
8939,overmore.com,LEADSCALE - Overmore Redirect WE HAVE REBRANDE...,"[(Personalized_marketing, 0.004480181588095144...","[Overmore Redirect, Click here Linkedin, leads...",Overmore Redirect Click here Linkedin leadscal...
8940,velocity-it.com,Digital Transformation & IT Automation | Veloc...,"[(Personalized_marketing, 0.004342529204348584...","[Business Process Automation, Robotic Process ...",Business Process Automation Robotic Process Au...
8941,synkd.life,"Synkd If you're seeing this message, that mean...","[(Personalized_marketing, 0.004428555070855129...","[app work, make this app, Synkd, message, brow...",app work make this app Synkd message browser w...
8942,zymplify.com,"Zymplify - Revenue Acceleration, Operations & ...","[(Personalized_marketing, 0.004283163887618378...","[Revenue Operations Marketing, Intent Revenue ...",Revenue Operations Marketing Intent Revenue Op...


In [25]:
df


,Vendor Name,Homepage Text,Predicted Category,Extracted Keywords,Extracted Keywords String
0,prnews.io,PRNEWS.IO — Content Marketing Platform With To...,"[(Personalized_marketing, 0.004315417412239644...","[Store Blog Sign, Version Online Store, conten...",Store Blog Sign Version Online Store content o...
1,proficrm.com,"CRM Software - Easy to use, modern and efficie...","[(Personalized_marketing, 0.004448535209032287...","[confirmation link, Login SIGNUP FREE, CRM Sof...",confirmation link Login SIGNUP FREE CRM Softwa...
2,atollon.com,Home | Atollon Přejít k hlavnímu obsahu Platfo...,"[(Personalized_marketing, 0.004403705303672669...","[Blog Cena Kontakt, Cena Kontakt Select, Home ...",Blog Cena Kontakt Cena Kontakt Select Home CRM...
3,dinamikcrm.com,İşinizi daha kolay ve verimli yönetmek istemez...,"[(Personalized_marketing, 0.004415895102354044...","[yönetmek istemez misiniz, verimli yönetmek is...",yönetmek istemez misiniz verimli yönetmek iste...
4,nectarcrm.com.br,Nectar | Software CRM para Gestão de Vendas B2...,"[(Personalized_marketing, 0.004387197681973473...","[Grátis Menu Menu, Teste Grátis Menu, Entrar T...",Grátis Menu Menu Teste Grátis Menu Entrar Test...
...,...,...,...,...,...
8939,overmore.com,LEADSCALE - Overmore Redirect WE HAVE REBRANDE...,"[(Personalized_marketing, 0.004480181588095144...","[Overmore Redirect, Click here Linkedin, leads...",Overmore Redirect Click here Linkedin leadscal...
8940,velocity-it.com,Digital Transformation & IT Automation | Veloc...,"[(Personalized_marketing, 0.004342529204348584...","[Business Process Automation, Robotic Process ...",Business Process Automation Robotic Process Au...
8941,synkd.life,"Synkd If you're seeing this message, that mean...","[(Personalized_marketing, 0.004428555070855129...","[app work, make this app, Synkd, message, brow...",app work make this app Synkd message browser w...
8942,zymplify.com,"Zymplify - Revenue Acceleration, Operations & ...","[(Personalized_marketing, 0.004283163887618378...","[Revenue Operations Marketing, Intent Revenue ...",Revenue Operations Marketing Intent Revenue Op...


In [27]:
df_out = []
file = open('g5g.csv', 'w+', newline ='')
import csv
for vendor_name,predictions in zip(df["Vendor Name"], df["Predicted Category"]):
    #print(vendor_name,predictions)
    for prediction in predictions:
        df_out.append([vendor_name,prediction[0],prediction[1]])
        #print(df_out)

with file:   
    write = csv.writer(file)
    write.writerows(df_out)       

In [ ]:
import gspread
import pandas as pd
from oauth2client.service_account 
import ServiceAccountCredentials